In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

# Load environment variables from the .env file
load_dotenv()

# Get the MySQL credentials from environment variables
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
dbname = os.getenv('MYSQL_DB')

# Create the database URL
db_url = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{dbname}"

# Create an engine instance
try:
    engine = create_engine(db_url)
    connection = engine.connect()
    print("Connected to the database successfully")
    connection.close()
except Exception as e:
    print(f"Error: {e}")


Connected to the database successfully
